In [2]:
##드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct 21 06:36:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [5]:
!wget https://thunlp.oss-cn-qingdao.aliyuncs.com/BERT_CCPoem_v1.zip
!unzip BERT_CCPoem_v1.zip

--2024-10-21 06:36:08--  https://thunlp.oss-cn-qingdao.aliyuncs.com/BERT_CCPoem_v1.zip
Resolving thunlp.oss-cn-qingdao.aliyuncs.com (thunlp.oss-cn-qingdao.aliyuncs.com)... 47.104.38.189
Connecting to thunlp.oss-cn-qingdao.aliyuncs.com (thunlp.oss-cn-qingdao.aliyuncs.com)|47.104.38.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150351681 (143M) [application/zip]
Saving to: ‘BERT_CCPoem_v1.zip’

BERT_CCPoem_v1.zip  100%[===================>] 143.39M  62.4MB/s    in 2.3s    

2024-10-21 06:36:11 (62.4 MB/s) - ‘BERT_CCPoem_v1.zip’ saved [150351681/150351681]

Archive:  BERT_CCPoem_v1.zip
   creating: BERT_CCPoem_v1/
  inflating: __MACOSX/._BERT_CCPoem_v1  
  inflating: BERT_CCPoem_v1/tokenizer_config.json  
  inflating: BERT_CCPoem_v1/special_tokens_map.json  
  inflating: BERT_CCPoem_v1/config.json  
  inflating: BERT_CCPoem_v1/training_args.bin  
  inflating: BERT_CCPoem_v1/vocab.txt  
  inflating: BERT_CCPoem_v1/pytorch_model.bin  


In [6]:
!pip install torch
!pip install transformers

In [7]:
import os
import logging

import torch
import torch.nn as nn
import numpy as np
from transformers import BertModel, BertTokenizer

logging.basicConfig(
    format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s', level=logging.INFO)

gpu_list = None


class Bert(nn.Module):
    def __init__(self, BERT_PATH='./BERT_CCPoem_v1'):
        super(Bert, self).__init__()

        self.bert = BertModel.from_pretrained(BERT_PATH)

    def init_multi_gpu(self, device):
        self.bert = nn.DataParallel(self.bert, device_ids=device)

    def forward(self, data, cls=False):
        result = []
        # print(data)
        x = data['input_ids']
        y = self.bert(x, attention_mask=data['attention_mask'],
                         token_type_ids=data['token_type_ids'])[0]

        if(cls):
            result = y[:, 0, :].view(y.size(0), -1)
            result = result.cpu().tolist()
        else:
            result = []
            y = y.cpu()
            # y = torch.mean(y, 1)
            # result = y.cpu().tolist()
            for i in range(y.shape[0]):
                tmp = y[i][1:torch.sum(data['attention_mask'][i]) - 1, :]
                result.append(tmp.mean(0).tolist())

        return result


class BertFormatter():
    def __init__(self, BERT_PATH='./BERT_CCPoem_v1'):
        self.tokenizer = BertTokenizer.from_pretrained(BERT_PATH)

    def process(self, data):
        res_dict = self.tokenizer.batch_encode_plus(
            data, pad_to_max_length=True)

        input_list = {'input_ids': torch.LongTensor(res_dict['input_ids']),
                      'attention_mask': torch.LongTensor(res_dict['attention_mask']),
                      "token_type_ids": torch.LongTensor(res_dict['token_type_ids'])}
        return input_list


def init(BERT_PATH="./BERT_CCPoem_v1"):
    global gpu_list
    gpu_list = []

    device_list = os.environ.get("CUDA_VISIBLE_DEVICES", "").split(",")
    if(device_list[0] == ""):
        device_list = []
    for a in range(0, len(device_list)):
        gpu_list.append(int(a))

    cuda = torch.cuda.is_available()
    logging.info("CUDA available: %s" % str(cuda))
    if not cuda and len(gpu_list) > 0:
        logging.error("CUDA is not available but specific gpu id")
        raise NotImplementedError

    model = Bert(BERT_PATH)
    formatter = BertFormatter(BERT_PATH)
    if len(gpu_list) > 0:
        model = model.cuda()
    if(len(gpu_list) > 1):
        try:
            model.init_multi_gpu(gpu_list)
        except Exception as e:
            logging.warning(
                "No init_multi_gpu implemented in the model, use single gpu instead. {}".format(str(e)))
    return model, formatter


def predict_vec_rep(data, model, formatter):
    data = formatter.process(data)
    model.eval()

    for i in data:
        if(isinstance(data[i], torch.Tensor)):
            if len(gpu_list) > 0:
                data[i] = data[i].cuda()

    result = model(data)

    return result


def cos_sim(vector_a, vector_b, sim=True):

    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    # num = float(vector_a * vector_b.T)

    num=float((vector_a * vector_b.T)[0, 0])
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    if(not sim):
        return cos
    sim = 0.5 + 0.5 * cos
    return sim

두보와 황정견 시구 비교를 위한 데이터 불러오기

In [8]:
!gdown 1ixLVPONoMFVdHJBtmKmjxNUGqk1w3keu -O 두보DB.xlsx

Downloading...
From: https://drive.google.com/uc?id=1ixLVPONoMFVdHJBtmKmjxNUGqk1w3keu
To: /content/두보DB.xlsx
100% 755k/755k [00:00<00:00, 76.0MB/s]


In [9]:
!gdown 1GAlPNmYEK-CFZxUSsBzjuIpbraaI7XeL -O 황정견DB.xlsx

Downloading...
From: https://drive.google.com/uc?id=1GAlPNmYEK-CFZxUSsBzjuIpbraaI7XeL
To: /content/황정견DB.xlsx
100% 786k/786k [00:00<00:00, 161MB/s]


In [10]:
import pandas as pd
import tqdm

DU_data=pd.read_excel("두보DB.xlsx",header=None).iloc[:,:2]
HUANG_data=pd.read_excel("황정견DB.xlsx",header=None).iloc[:,:2]

In [11]:
DU_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19478 entries, 0 to 19477
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       19478 non-null  object
 1   1       19478 non-null  object
dtypes: object(2)
memory usage: 304.5+ KB


두보와 황정견 데이터 비교

In [12]:
###주의 문구 제거

logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

In [13]:
#왜 같다고 뜨지??
model, formatter = init()
result_0 = predict_vec_rep(["润础闹苍藓"], model, formatter)[0] ##원래 문장
model, formatter = init()
result_1 = predict_vec_rep(['盐车虽绊骥'], model, formatter)[0] #비교대상 1
model, formatter = init()
result_2 = predict_vec_rep(["词华倾后辈"], model, formatter)[0] #비교대상 2

print(cos_sim(result_0, result_1))
print(cos_sim(result_0, result_2))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0.6546517035781813
0.6219595932448706


In [14]:
##구글드라이브에 파일이 잘써지는지 확인

with open('/content/drive/My Drive/버트결과/test1.txt', 'w') as f: f.write('Hello, Google Drive!')

In [15]:
def process_batch(start_idx, end_idx):
    # 두보 시구 갯수 지정
    batch_size = end_idx - start_idx
    batch_df = pd.DataFrame(columns=["황정견", "두보", "코사인 유사도"])

    # 비교는 간체로 하되, 결과표시는 번체로.
    model, formatter = init()  # 루프 외부에서 한 번만 실행

    for i in tqdm.tqdm(range(batch_size)):
        #황정견 번체 시구
        huang_traditional = HUANG_data.iloc[start_idx + i, 0]
        #황정견 간체 시구
        huang_simplified = HUANG_data.iloc[start_idx + i, 1]

        huang_predict = predict_vec_rep([huang_simplified], model, formatter)[0]

        print(f"{huang_traditional}({huang_simplified})와 나머지 시구를 비교중..")

        for j in range(len(DU_data)):

            #두보 번체 시구
            du_traditional = DU_data.iloc[j, 0]
            #두보 간체 시구
            du_simplified = DU_data.iloc[j, 1]

            du_predict = predict_vec_rep([du_simplified], model, formatter)[0]
            res = cos_sim(huang_predict, du_predict)
            if res > 0.8:
                batch_df.loc[len(batch_df)] = [huang_traditional, du_traditional, res]

    #파일 저장...
    file_path = f'/content/drive/My Drive/버트결과/황정견의 {int(start_idx)+1}-{int(end_idx)}번 시구와 두보의 모든 시구 비교.csv'
    batch_df.to_csv(file_path, index=False)
    print("파일 저장 완료!")
    return batch_df

In [16]:
import warnings

# FutureWarning을 무시
warnings.filterwarnings("ignore", category=FutureWarning)


\

In [17]:
print("hello world!")

hello world!


In [ ]:
process_batch(18100,18200)

  0%|          | 0/100 [00:00<?, ?it/s]

休泥他家晚菊叢(休泥他家晚菊丛)와 나머지 시구를 비교중..


  1%|          | 1/100 [04:52<8:03:04, 292.77s/it]

顧笑千金延客醉(顾笑千金延客醉)와 나머지 시구를 비교중..


  2%|▏         | 2/100 [09:38<7:51:35, 288.73s/it]

...

 99%|█████████▉| 99/100 [7:55:13<04:45, 285.94s/it]

舊贈恐能開寶匣(旧赠恐能开宝匣)와 나머지 시구를 비교중..


100%|██████████| 100/100 [7:59:58<00:00, 287.99s/it]

파일 저장 완료!


,황정견,두보,코사인 유사도
0,休泥他家晚菊叢,時菊碎榛叢,0.817887
1,顧笑千金延客醉,百罰深杯辭不辭,0.801879
2,顧笑千金延客醉,客醉揮金碗,0.828817
3,顧笑千金延客醉,未覺千金滿高價,0.801434
4,顧笑千金延客醉,縱酒欲謀良夜醉,0.808977
...,...,...,...
889,窄袖春衫甘草黄,黄帽青鞋歸去來,0.804095
890,窄袖春衫甘草黄,春草隨青袍,0.854383
891,舊贈恐能開寶匣,脱劍主人贈,0.809110
892,舊贈恐能開寶匣,塵匣元開鏡,0.808669
